In [57]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.spatial import distance_matrix
from sklearn import preprocessing
import os
from sklearn.utils import shuffle

In [28]:
base_path = '/home/jovyan/work/Molecular_Properties'

In [29]:
train = pd.read_csv(base_path + '/train.csv')
test = pd.read_csv(base_path + '/test.csv')
structures = pd.read_csv(base_path + '/structures.csv')

In [30]:
train_bonds = pd.read_csv(base_path + '/train_bonds.csv')
test_bonds = pd.read_csv(base_path + '/test_bonds.csv')

In [31]:
angs = pd.read_csv(base_path + "/angles.csv")

In [32]:
angs.head()

,molecule_name,atom_index_0,atom_index_1,shortest_path_atoms,shortest_path_n_bonds,cosinus,dihedral
0,dsgdb9nsd_000001,1,0,NaN,1,NaN,NaN
1,dsgdb9nsd_000001,1,2,C,2,-0.33329,NaN
2,dsgdb9nsd_000001,1,3,C,2,-0.33333,NaN
3,dsgdb9nsd_000001,1,4,C,2,-0.33335,NaN
4,dsgdb9nsd_000001,2,0,NaN,1,NaN,NaN


In [33]:
scale_min  = train['scalar_coupling_constant'].min()
scale_max  = train['scalar_coupling_constant'].max()
scale_mid = (scale_max + scale_min)/2
scale_norm = scale_max - scale_mid

train['scalar_coupling_constant'] = (train['scalar_coupling_constant'] - scale_mid)/scale_norm

# One hot encoding gets  too big for Kaggle, let's try label
# use npz now, back to OH
train[['1JHC', '1JHN', '2JHC', '2JHH', '2JHN', '3JHC', '3JHH', '3JHN']] =  pd.get_dummies(train['type'])
test[['1JHC', '1JHN', '2JHC', '2JHH', '2JHN', '3JHC', '3JHH', '3JHN']]  =  pd.get_dummies(test['type'])

#le = preprocessing.LabelEncoder()
#le.fit(['1JHC', '1JHN', '2JHC', '2JHH', '2JHN', '3JHC', '3JHH', '3JHN'])
#train['l_type'] = (le.transform(train['type']) + 1)/8.
#test['l_type'] = (le.transform(test['type']) + 1)/8.

In [34]:
structures[['C', 'F' ,'H', 'N', 'O']] = pd.get_dummies(structures['atom'])
structures[['x', 'y', 'z']] = structures[['x', 'y', 'z']]/10.

In [35]:
structures.head()

,molecule_name,atom_index,atom,x,y,z,C,F,H,N,O
0,dsgdb9nsd_000001,0,C,-0.001270,0.108580,0.000800,1,0,0,0,0
1,dsgdb9nsd_000001,1,H,0.000215,-0.000603,0.000198,0,0,1,0,0
2,dsgdb9nsd_000001,2,H,0.101173,0.146375,0.000028,0,0,1,0,0
3,dsgdb9nsd_000001,3,H,-0.054082,0.144753,-0.087664,0,0,1,0,0
4,dsgdb9nsd_000001,4,H,-0.052381,0.143793,0.090640,0,0,1,0,0


In [36]:
test_bonds[['nbond_1', 'nbond_1.5', 'nbond_2', 'nbond_3']] = pd.get_dummies(test_bonds['nbond'])#test_bonds['nbond']/3
train_bonds[['nbond_1', 'nbond_1.5', 'nbond_2', 'nbond_3']] = pd.get_dummies(train_bonds['nbond'])#train_bonds['nbond']/3

In [37]:
train_bonds.head()

,molecule_name,atom_index_0,atom_index_1,nbond,L2dist,error,bond_type,nbond_1,nbond_1.5,nbond_2,nbond_3
0,dsgdb9nsd_000001,0,1,1.0,1.091953,0,1.0CH,1,0,0,0
1,dsgdb9nsd_000001,0,2,1.0,1.091952,0,1.0CH,1,0,0,0
2,dsgdb9nsd_000001,0,3,1.0,1.091946,0,1.0CH,1,0,0,0
3,dsgdb9nsd_000001,0,4,1.0,1.091948,0,1.0CH,1,0,0,0
4,dsgdb9nsd_000002,0,1,1.0,1.017190,0,1.0HN,1,0,0,0


In [38]:
angs['dihedral'] = angs['dihedral']/np.pi
# Should I rather one-hot this?
angs['shortest_path_n_bonds'] = angs['shortest_path_n_bonds']/6.0
angs = angs.fillna(0)

In [39]:
train_mol_names = train['molecule_name'].unique()
test_mol_names  = test['molecule_name'].unique()

train_structures = structures.loc[structures['molecule_name'].isin(train_mol_names)]
test_structures = structures.loc[structures['molecule_name'].isin(test_mol_names)]

train_struct_group = train_structures.groupby('molecule_name')
test_struct_group  = test_structures.groupby('molecule_name')

train_group = train.groupby('molecule_name')
test_group  = test.groupby('molecule_name')

train_bond_group = train_bonds.groupby('molecule_name')
test_bond_group  = test_bonds.groupby('molecule_name')

train_angs = angs.loc[angs['molecule_name'].isin(train_mol_names)]
test_angs = angs.loc[angs['molecule_name'].isin(test_mol_names)]

train_angs_group = train_angs.groupby('molecule_name')
test_angs_group  = test_angs.groupby('molecule_name')

# Find max nodes in graph:
max_size = train_struct_group.size().max()

In [40]:
# Values our nodes will have
node_vals = ['C', 'F' ,'H', 'N', 'O']#, 'x', 'y', 'z']
#Values our edges will have (minus distance, for now)
bond_vals = ['nbond_1', 'nbond_1.5', 'nbond_2', 'nbond_3']#['nbond']
j_coup_vals = ['1JHC', '1JHN', '2JHC', '2JHH', '2JHN', '3JHC', '3JHH', '3JHN']#'l_type']
ang_vals = ['shortest_path_n_bonds','cosinus','dihedral']
edge_vals = j_coup_vals + bond_vals + ang_vals

# Find amount of training molecules
n_train_mols = len(train_mol_names)
n_test_mols = len(test_mol_names)

# Find dim of edges and nodes
bond_dim  = len(bond_vals)
j_coup_dim= len(j_coup_vals)
ang_dim   = len(ang_vals)
node_dim  = len(node_vals)
edge_dim  = len(edge_vals) 

# Additional edge dims for distances 
add_edge_dim = 1

In [41]:
train_nodes_array     = np.zeros((n_train_mols, max_size, node_dim), dtype=np.float32) 
train_in_edges_array  = np.zeros((n_train_mols, max_size, max_size, edge_dim + add_edge_dim),dtype=np.float32) 
train_out_edges_array = np.zeros((n_train_mols, max_size, max_size, 1),dtype=np.float32) 

test_nodes_array     = np.zeros((n_test_mols, max_size, node_dim), dtype=np.float32) 
test_in_edges_array  = np.zeros((n_test_mols, max_size, max_size, edge_dim + add_edge_dim),dtype=np.float32) 

In [42]:
def make_arrs(val_group, struct_group, bond_group, ang_group, test):
    i = 0
    for values, structs, bonds, angles in zip(val_group, struct_group, bond_group, ang_group):
        if (not i%1000):
            print(i)

        # Calculate distances
        distances = np.zeros((max_size, max_size, 1))
        coords = structs[1][['x','y','z']].values
        dists  = distance_matrix(coords, coords)
        distances[:dists.shape[0],:dists.shape[1], 0] = dists 
        
        # Create nodes
        mol_info = structs[1][node_vals].values
        nodes = np.zeros((max_size, node_dim))
        nodes[:mol_info.shape[0], :mol_info.shape[1]] = mol_info

        # Create edges
        in_feats = np.zeros((max_size, max_size, j_coup_dim))
        ind = values[1][['atom_index_0', 'atom_index_1' ]].values
        in_feats[ind[:,0], ind[:,1], 0:j_coup_dim] = values[1][j_coup_vals].values
        in_feats[ind[:,1], ind[:,0], 0:j_coup_dim] = in_feats[ind[:,0], ind[:,1], 0:j_coup_dim]

        # Create bonds
        in_bonds = np.zeros((max_size, max_size, bond_dim))
        ind_bonds = bonds[1][['atom_index_0', 'atom_index_1' ]].values
        in_bonds[ind_bonds[:,0], ind_bonds[:,1]] = bonds[1][bond_vals].values
        in_bonds[ind_bonds[:,1], ind_bonds[:,0]] = in_bonds[ind_bonds[:,0], ind_bonds[:,1]]
        
        # Create angles
        ind_angs = angles[1][['atom_index_0', 'atom_index_1' ]].values
        ang_mat  = np.zeros((max_size, max_size, ang_dim))
        ang_mat[ind_angs[:,0], ind_angs[:,1]]  = angles[1][ang_vals]
        ang_mat[ind_angs[:,1], ind_angs[:,0]]  = ang_mat[ind_angs[:,0], ind_angs[:,1]]
        
        # concat all edge values 
        in_edges = np.concatenate((in_feats, in_bonds, ang_mat, distances),axis=2)
        
        if not test:           
            out_edges = np.zeros((max_size, max_size, 1))
            out_edges[ind[:,0], ind[:,1], 0] = values[1]['scalar_coupling_constant' ].values
            out_edges[ind[:,1], ind[:,0], 0] = out_edges[ind[:,0], ind[:,1], 0]
        

            train_nodes_array[i]      = nodes
            train_in_edges_array[i]   = in_edges
            train_out_edges_array[i]  = out_edges
        else:
            test_nodes_array[i]      = nodes
            test_in_edges_array[i]   = in_edges
        i = i + 1

In [43]:
make_arrs(train_group, train_struct_group, train_bond_group, train_angs_group, test = False)

0
1000
2000


In [44]:
train_group

In [24]:
make_arrs(test_group, test_struct_group, test_bond_group, test_angs_group, test = True)

0
1000
2000


In [50]:
len(test_in_edges_array)

45772

In [51]:
np.savez_compressed("nodes_train.npz" , train_nodes_array)
np.savez_compressed("in_edges_train.npz" , train_in_edges_array)
np.savez_compressed("out_edges_train.npz" , train_out_edges_array)
np.savez_compressed("nodes_test.npz" , test_nodes_array)
np.savez_compressed("in_edges_test.npz" , test_in_edges_array)

In [52]:
################### Modeling Part! GPU instance required ##############################

nodes_train     = np.load("nodes_train.npz" )['arr_0']
in_edges_train  = np.load("in_edges_train.npz")['arr_0']
out_edges_train = np.load("out_edges_train.npz" )['arr_0']
nodes_test     = np.load("nodes_test.npz" )['arr_0']
in_edges_test  = np.load("in_edges_test.npz")['arr_0']

In [53]:
out_labels = out_edges_train.reshape(-1,out_edges_train.shape[1]*out_edges_train.shape[2],1)
in_edges_train = in_edges_train.reshape(-1,in_edges_train.shape[1]*in_edges_train.shape[2],in_edges_train.shape[3])
in_edges_test  = in_edges_test.reshape(-1,in_edges_test.shape[1]*in_edges_test.shape[2],in_edges_test.shape[3])

In [58]:
nodes_train, in_edges_train, out_labels = shuffle(nodes_train, in_edges_train, out_labels)

In [61]:
len(nodes_train)

85003